# Analise Comparativa

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('../data/raw/data.csv')
dictionary_df = pd.read_csv('../data/external/dictionary.csv')

In [3]:

# Filtering variables based on criteria
target_variable = 'price'

useless_variables = (
    dictionary_df
    .query("tipo == 'inútil'")
    .variavel
    .to_list()
)

nominal_variables = (
    dictionary_df
    .query("subtipo == 'nominal' and variavel != @target_variable")
    .variavel
    .to_list()
)

ordinal_variables = (
    dictionary_df
    .query("subtipo == 'ordinal' and variavel != @target_variable")
    .variavel
    .to_list()
)

continuous_variables = (
    dictionary_df
    .query("subtipo == 'contínua' and variavel != @target_variable")
    .variavel
    .to_list()
)

discrete_variables = (
    dictionary_df
    .query("subtipo == 'discreta' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Creating X and y datasets
X = df.drop(columns=[target_variable] + useless_variables)
y = df[target_variable]

{
    "X_shape": X.shape,
    "y_shape": y.shape,
    "Nominal Variables": nominal_variables,
    "Ordinal Variables": ordinal_variables,
    "Continuous Variables": continuous_variables,
    "Discrete Variables": discrete_variables
}

{'X_shape': (53940, 10),
 'y_shape': (53940,),
 'Nominal Variables': [],
 'Ordinal Variables': ['cut', 'color', 'clarity'],
 'Continuous Variables': ['carat', 'depth', 'table', 'x', 'y', 'z'],
 'Discrete Variables': []}

In [4]:

nominal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), 
    ("encoding", OneHotEncoder(sparse_output=False, handle_unknown='infrequent_if_exist'))
])

ordinal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), 
    ("encoding", OrdinalEncoder())
])

continuous_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')), 
    ("scaling", StandardScaler())
])

discrete_preprocessor = Pipeline(steps=[
    ("missing", KNNImputer()), 
    ("scaling", StandardScaler())
])


In [5]:
# Combine preprocessors in a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("nominal", nominal_preprocessor, nominal_variables),
        ("ordinal", ordinal_preprocessor, ordinal_variables),
        ("continuous", continuous_preprocessor, continuous_variables),
        ("discrete", discrete_preprocessor, discrete_variables),
    ]
)

In [6]:
models = [DummyRegressor(strategy='mean'), LinearRegression(), KNeighborsRegressor(n_neighbors=5)]
metrics = [
    'neg_mean_absolute_error',
    'neg_mean_squared_error',
    'neg_mean_absolute_percentage_error',
    'r2',
]
monte_carlo = ShuffleSplit(n_splits=10, test_size=.2, random_state=42)
# hold_out = ShuffleSplit(n_splits=1, test_size=.2, random_state=42)
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [7]:
results_total = None
for model in models:
    model_name = model.__class__.__name__
    print(f"rodando para o modelo: {model_name}")
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model),
    ])

    scores = cross_validate(
        approach, X, y,
        scoring=metrics, 
        cv=monte_carlo
    )
    results_model = pd.DataFrame(scores)
    results_model['model'] = model_name
    if results_total is None:
        results_total = results_model
    else:
        results_total = pd.concat([results_total, results_model])

rodando para o modelo: DummyRegressor
rodando para o modelo: LinearRegression
rodando para o modelo: KNeighborsRegressor


In [8]:
results_total.groupby('model').agg(['mean', 'std']).T

model                                         DummyRegressor  \
fit_time                                mean    4.266479e-02   
                                        std     1.094923e-02   
score_time                              mean    9.534454e-03   
                                        std     1.441761e-03   
test_neg_mean_absolute_error            mean   -3.022423e+03   
                                        std     1.702945e+01   
test_neg_mean_squared_error             mean   -1.582750e+07   
                                        std     2.631958e+05   
test_neg_mean_absolute_percentage_error mean   -1.877466e+00   
                                        std     1.333744e-02   
test_r2                                 mean   -5.216110e-05   
                                        std     5.702326e-05   

model                                         KNeighborsRegressor  \
fit_time                                mean             0.087072   
                                        std              0.013518   
score_time                              mean             0.275957   
                                        std              0.017870   
test_neg_mean_absolute_error            mean          -389.101696   
                                        std              4.203031   
test_neg_mean_squared_error             mean       -557133.098198   
                                        std          24072.413778   
test_neg_mean_absolute_percentage_error mean            -0.109717   
                                        std              0.001017   
test_r2                                 mean             0.964789   
                                        std              0.001650   

model                                         LinearRegression  
fit_time                                mean      7.966759e-02  
                                        std       1.710426e-02  
score_time                              mean      1.519840e-02  
                                        std       3.669444e-03  
test_neg_mean_absolute_error            mean     -8.609442e+02  
                                        std       7.631357e+00  
test_neg_mean_squared_error             mean     -1.814252e+06  
                                        std       5.911209e+04  
test_neg_mean_absolute_percentage_error mean     -3.839988e-01  
                                        std       3.842612e-03  
test_r2                                 mean      8.853430e-01  
                                        std       4.041712e-03